<a href="https://colab.research.google.com/github/imai-ry/colabo/blob/main/AISprediction20210601.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AIS進路予測
hamakanaya-ais.log-161

# データ読み込み

In [1]:
# AISデコードライブラリのインストール
!python3 -m pip install libais

# 地図表示ライブラリ
!python3 -m pip install folium

     |████████████████████████████████| 163kB 5.2MB/s 
  Created wheel for libais: filename=libais-0.17-cp37-cp37m-linux_x86_64.whl size=1041992 sha256=c21034addaef42358ecaae54cf2218058db4e9a5d9b239634ef1be5d238ac986
  Stored in directory: /root/.cache/pip/wheels/2f/bc/f8/902480382d20985b4c59b0d505d4897c25c0d64e3cba51df33
Successfully built libais


In [2]:
# AISデータの保存場所指定

HOME = '/content/drive/MyDrive/colab-data/'
path = HOME + "hamakanaya-ais.log-1617034621"
print(path)

/content/drive/MyDrive/colab-data/hamakanaya-ais.log-1617034621


In [3]:
# AISデータの読み込み

import pandas as pd

rtime1 = []
rtime2 = []
raw_msgs = []

with open(path) as f:
  sl = f.readlines()
  for s in sl:
    s = s.split('\t')
    rtime1.append(s[0])
    rtime2.append(s[1])
    raw_msgs.append(s[2].strip('\n'))

d = {'rtime1': rtime1, "rtime2": rtime2, "raw_msg": raw_msgs}
df = pd.DataFrame(data=d)
df

,rtime1,rtime2,raw_msg
0,2021:03:29:15:17:02:859946,2021:03:30:00:17:02:859946,"!BSVDM,2,1,4,B,56KJU`h2;0WD`OG??>0DUL60l59F0pv..."
1,2021:03:29:15:17:02:886122,2021:03:30:00:17:02:886122,"!BSVDM,2,2,4,B,08jT8556H88888888888880,2*72"
2,2021:03:29:15:17:03:068219,2021:03:30:00:17:03:068219,"!BSVDM,1,1,,A,16KPivP01P9wTd`D6WvH;Vb620S1,0*61"
3,2021:03:29:15:17:03:518695,2021:03:30:00:17:03:518695,"!BSVDM,1,1,,A,16K2:?00019wTM>D9lHpC6<80821,0*7F"
4,2021:03:29:15:17:03:704913,2021:03:30:00:17:03:704913,"!BSVDM,1,1,,B,16K2Wb001M9wJwJD66R8pG<80@28,0*55"
...,...,...,...
12578,2021:03:29:16:17:00:379724,2021:03:30:01:17:00:379724,"!BSVDM,1,1,,B,16K8Dr@0019wU;rD9lA8vo2200Rr,0*0C"
12579,2021:03:29:16:17:00:505373,2021:03:30:01:17:00:505373,"!BSVDM,1,1,,A,36K2=G@Oin9wf2DD5E=P`0Ml00u1,0*03"
12580,2021:03:29:16:17:00:531470,2021:03:30:01:17:00:531470,"!BSVDM,1,1,,A,16K2<Ph01T9wbAdD4kHWJ6220<2r,0*2D"
12581,2021:03:29:16:17:00:640902,2021:03:30:01:17:00:640902,"!BSVDM,1,1,,A,16K2twh01t9wlftD:GEG05cn0D7I,0*56"


In [4]:
# AISのデコード
%cd $HOME
from ais_decoder.ais_decoder import AisDecoder
import numpy as np

decoder = AisDecoder(["!BSVDM"])
nav_data = {
    "id": [],
    "mmsi": [],
    "sog": [],
    "x": [],
    "y": [],
    "cog": [],
    "true_heading": [],
}

for raw_msg in df["raw_msg"]:
    out = decoder.decode(raw_msg)
    decoded = out["decoded"]
    if not decoded:
        continue
    for key in nav_data.keys():
        val = decoded[key] if (key in decoded) else np.nan
        nav_data[key].append(val) 

ais_df = pd.DataFrame(data=nav_data)
ais_df


/content/drive/MyDrive/colab-data


,id,mmsi,sog,x,y,cog,true_heading
0,5,431400355,NaN,NaN,NaN,NaN,NaN
1,1,431501818,9.6,139.716940,35.133428,209.399994,213.0
2,1,431000124,0.1,139.716118,35.220645,212.399994,198.0
3,1,431007656,9.3,139.683808,35.119160,227.300003,230.0
4,1,431000742,0.1,139.713855,35.222645,351.600006,352.0
...,...,...,...,...,...,...,...
10932,1,431101161,0.1,139.718608,35.220593,229.899994,225.0
10933,3,431000925,11.8,139.748817,35.098117,16.000000,14.0
10934,1,431000707,10.0,139.735983,35.083683,189.600006,193.0
10935,1,431013119,12.4,139.771677,35.235555,179.199997,181.0


In [5]:
# 欠損値NaN除去
ais_df = ais_df.dropna(how="any")

# MMSI別に抽出
targets = [ais_df[ais_df["mmsi"]==mmsi] for mmsi in ais_df["mmsi"].unique()]
targets[0]

,id,mmsi,sog,x,y,cog,true_heading
1,1,431501818,9.6,139.716940,35.133428,209.399994,213.0
18,1,431501818,9.6,139.716670,35.133048,210.600006,213.0
34,1,431501818,9.7,139.716417,35.132702,210.699997,212.0
52,1,431501818,9.7,139.716180,35.132343,209.100006,212.0
79,1,431501818,9.7,139.715882,35.131917,210.300003,213.0
...,...,...,...,...,...,...,...
8135,1,431501818,9.4,139.594795,35.068432,243.100006,245.0
8500,1,431501818,9.4,139.588982,35.066153,245.199997,244.0
8536,1,431501818,9.3,139.588552,35.065990,245.300003,246.0
8881,1,431501818,9.4,139.583767,35.064130,244.300003,245.0


In [6]:
# target0を例に図示
# xyのみ抽出
xy = targets[0].loc[:, ["x","y"]]

# 緯度経度を入れ替えて、pythonリストに変換
latlngs = xy.values[:,[1,0]].tolist()
latlngs[:5]

# 航跡の表示
import folium
m = folium.Map(location=[35.3, 140], zoom_start=10)
polyline = folium.PolyLine(latlngs, color="red")
polyline.add_to(m)
m

# データセットの作成

In [8]:
test_target = targets[0]
targets.pop(0)

training_size = round(len(targets) * 0.7)
test_size = len(targets) - training_size
timesteps = 10
n_features = 2

# x,y抽出
train_x = np.zeros((0,timesteps,n_features))
train_y = np.zeros((0,n_features))

for i in range(len(targets)):
  x = targets[i]["x"].tolist()
  y = targets[i]["y"].tolist()
  if(len(x)<timesteps+1):
    continue
  state = np.array([x,y]).T
  n_trajectory_points, n_features = state.shape
  for i in range(n_trajectory_points-timesteps):
    tx = state[i:i+timesteps, :]
    ty = state[i+timesteps, :]
    train_x = np.append(train_x, tx.reshape(1,tx.shape[0],tx.shape[1]), axis=0)
    train_y = np.append(train_y, ty.reshape(1,ty.shape[0]), axis=0)


print(f"船の総数：{len(targets)}")
print(f"training size: {training_size}")
print(f"test size: {test_size}")
print(f"timesteps: {timesteps}")
print(f"train_x.shape:{train_x.shape}, train_y.shape:{train_y.shape}")

print(test_target)

"""
print(train_x[0,:,:])
print(train_y[0,:])
print(train_x[1,:,:])
print(train_y[1,:])
"""

船の総数：123
training size: 86
test size: 37
timesteps: 10
train_x.shape:(8070, 10, 2), train_y.shape:(8070, 2)
       id       mmsi  sog           x          y         cog  true_heading
2       1  431000124  0.1  139.716118  35.220645  212.399994         198.0
19      1  431000124  0.1  139.716118  35.220647  212.399994         198.0
36      1  431000124  0.1  139.716118  35.220642  212.399994         198.0
81      1  431000124  0.1  139.716113  35.220640  212.399994         198.0
117     1  431000124  0.1  139.716113  35.220643  212.399994         198.0
...    ..        ...  ...         ...        ...         ...           ...
10738   1  431000124  0.1  139.716117  35.220643  212.399994         197.0
10767   1  431000124  0.1  139.716117  35.220643  212.399994         197.0
10792   1  431000124  0.1  139.716117  35.220643  212.399994         197.0
10824   1  431000124  0.1  139.716117  35.220643  212.399994         197.0
10887   1  431000124  0.1  139.716113  35.220643  212.399994       

'\nprint(train_x[0,:,:])\nprint(train_y[0,:])\nprint(train_x[1,:,:])\nprint(train_y[1,:])\n'

# モデルの定義と学習

In [9]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM

# モデル構築
units = 30
model = Sequential([
  LSTM(units, input_shape=(timesteps, n_features), return_sequences=False),
  Dense(n_features, activation="linear")      
])
model.compile(loss="mean_squared_error", optimizer="adam")
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30)                3960      
_________________________________________________________________
dense (Dense)                (None, 2)                 62        
Total params: 4,022
Trainable params: 4,022
Non-trainable params: 0
_________________________________________________________________


In [11]:
train_x.shape

(8070, 10, 2)

In [12]:
train_y.shape

(8070, 2)

In [16]:
train_x[:3,:,:]

array([[[139.68380833,  35.11916   ],
        [139.68349667,  35.118925  ],
        [139.68301167,  35.11858167],
        [139.68260833,  35.11828833],
        [139.68228333,  35.11804833],
        [139.68190167,  35.11774667],
        [139.68143   ,  35.11741667],
        [139.680655  ,  35.11685   ],
        [139.68027   ,  35.116575  ],
        [139.67991833,  35.116315  ]],

       [[139.68349667,  35.118925  ],
        [139.68301167,  35.11858167],
        [139.68260833,  35.11828833],
        [139.68228333,  35.11804833],
        [139.68190167,  35.11774667],
        [139.68143   ,  35.11741667],
        [139.680655  ,  35.11685   ],
        [139.68027   ,  35.116575  ],
        [139.67991833,  35.116315  ],
        [139.67956833,  35.116035  ]],

       [[139.68301167,  35.11858167],
        [139.68260833,  35.11828833],
        [139.68228333,  35.11804833],
        [139.68190167,  35.11774667],
        [139.68143   ,  35.11741667],
        [139.680655  ,  35.11685   ],
        

In [15]:
train_y[:3,:]

array([[139.67956833,  35.116035  ],
       [139.67909833,  35.11568167],
       [139.67876167,  35.11547   ]])

In [10]:
history = model.fit(train_x, train_y, epochs=10, validation_split=0.2)

Epoch 1/10
202/202 [==============================] - 4s 10ms/step - loss: 9497.4229 - val_loss: 9002.8262
Epoch 2/10
202/202 [==============================] - 1s 6ms/step - loss: 8593.2510 - val_loss: 8197.1621
Epoch 3/10
202/202 [==============================] - 1s 6ms/step - loss: 7862.5537 - val_loss: 7535.0513
Epoch 4/10
202/202 [==============================] - 1s 6ms/step - loss: 7232.0317 - val_loss: 6933.8115
Epoch 5/10
202/202 [==============================] - 1s 7ms/step - loss: 6655.6519 - val_loss: 6381.5527
Epoch 6/10
202/202 [==============================] - 1s 6ms/step - loss: 6124.7539 - val_loss: 5871.6079
Epoch 7/10
202/202 [==============================] - 1s 6ms/step - loss: 5633.5972 - val_loss: 5398.9351
Epoch 8/10
202/202 [==============================] - 1s 6ms/step - loss: 5177.5991 - val_loss: 4959.3638
Epoch 9/10
202/202 [==============================] - 1s 7ms/step - loss: 4752.9136 - val_loss: 4549.3638
Epoch 10/10
202/202 [========================

In [ ]:
# test target
test_x = np.zeros((0,timesteps,n_features))
test_y = np.zeros((0,n_features))
x = test_target["x"].tolist()
y = test_target["y"].tolist()
state = np.array([x,y]).T
n_trajectory_points, n_features = state.shape
for i in range(n_trajectory_points-timesteps):
  tx = state[i:i+timesteps, :]
  ty = state[i+timesteps, :]
  test_x = np.append(test_x, tx.reshape(1,tx.shape[0],tx.shape[1]), axis=0)
  test_y = np.append(test_y, ty.reshape(1,ty.shape[0]), axis=0)

pred = model.predict(test_x)
print(test_x.shape,test_y.shape)
print(pred.shape)
pre_xy = pred[:,:].tolist()
test_xy = test_y[:,:].tolist()

(218, 10, 2) (218, 2)
(218, 2)


In [ ]:
pre_xy = pred[:,:].tolist()
pre_latlngs = [[y,x] for x,y in pre_xy]
test_latlngs = [[y,x] for x,y in test_xy]
print(pre_latlngs[:10])
print(test_latlngs[:10])
print(len(pre_latlngs), len(test_latlngs))

[[35.14801788330078, 139.7270965576172], [35.14801788330078, 139.7270965576172], [35.14801788330078, 139.7270965576172], [35.14801788330078, 139.7270965576172], [35.14801788330078, 139.7270965576172], [35.14801788330078, 139.7270965576172], [35.14801788330078, 139.7270965576172], [35.14801788330078, 139.7270965576172], [35.14801788330078, 139.7270965576172], [35.14801788330078, 139.7270965576172]]
[[35.12956166666667, 139.71428], [35.12916, 139.714005], [35.12876333333333, 139.71373166666666], [35.128328333333336, 139.71343], [35.12792666666667, 139.71316333333334], [35.12761, 139.712955], [35.12717166666667, 139.71264833333333], [35.12678833333333, 139.71237833333333], [35.12639166666666, 139.71211833333334], [35.125955, 139.71183]]
218 218


In [ ]:
import folium
m = folium.Map(location=[35.3, 140], zoom_start=10)
pre_polyline = folium.PolyLine(pre_latlngs, color="red")
pre_polyline.add_to(m)
test_polyline = folium.PolyLine(test_latlngs, color="black")
test_polyline.add_to(m)
m

In [ ]:


# target0を例に図示
# xyのみ抽出
xy = targets[0].loc[:, ["x","y"]]

# 緯度経度を入れ替えて、pythonリストに変換
latlngs = xy.values[:,[1,0]].tolist()
latlngs[:5]

# 航跡の表示
import folium
m = folium.Map(location=[35.3, 140], zoom_start=10)
polyline = folium.PolyLine(latlngs, color="red")
polyline.add_to(m)
m